* https://software.intel.com/content/www/us/en/develop/articles/using-intel-mkl-in-your-python-programs.html
* https://software.intel.com/content/www/us/en/develop/documentation/mkl-tutorial-c/top/multiplying-matrices-using-dgemm.html

In [1]:
from ctypes import c_double, cdll, c_int, byref

def print_mat(mat, m, n):
    for i in range(m):
        row = []
        for j in range(n):
            row.append((mat[i*n+j]))
        print(row)
        print('-'*20)
            
def construct_array(arr):
    """convert a python list into c array.
    """
    n = len(arr)
    amat = c_double * n  # preallocate array of n elements, each a double
    return amat(*arr)

In [2]:
mkl = cdll.LoadLibrary("./libmkl_rt.dylib")
dgemm = mkl.cblas_dgemm  # mkl as a dgemm function
print(vars(mkl))

{'_name': './libmkl_rt.dylib', '_FuncPtr': <class 'ctypes.CDLL.__init__.<locals>._FuncPtr'>, '_handle': 140409870245616, 'cblas_dgemm': <_FuncPtr object at 0x1128ac2c0>}


In [3]:
# create 3 matrices
a = construct_array([1, 2, 3, 4, 5, 6])
b = construct_array([0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0])
c = construct_array([5, 1, 3, 3, 11, 4, 6, 9])
print_mat(c, 2, 4)
# print_mat(a, 2, 3) 
# print_mat(b, 3, 4)

[5.0, 1.0, 3.0, 3.0]
--------------------
[11.0, 4.0, 6.0, 9.0]
--------------------


In [4]:
"""
https://software.intel.com/content/www/us/en/develop/documentation/mkl-tutorial-c/top/multiplying-matrices-using-dgemm.html

cblas_dgemm(CblasRowMajor, CblasNoTrans, CblasNoTrans, m, n, k, alpha, A, k, B, n, beta, C, n)
"""
Order = 101  # 101 for row-major, 102 for column major data structures
TransA = 111 # 111 for no transpose, 112 for transpose, and 113 for conjugate transpose
TransB = 111
m = 2
n = 4
k = 3
alpha = 1.0  # scaling factor, will be converted to a c double
beta = -1.0
lda = k  # number of columns in matrix A, which is a m x k matrix
ldb = n  # number of columns in matrix B, which is a k x n matrix
ldc = n  # number of columns in matrix C, which is a m x n matrix 

In [5]:
# call mkl's dgemm method, result is stored in pointer to matrix C

dgemm(c_int(Order), c_int(TransA), c_int(TransB), 
      c_int(m), c_int(n), c_int(k), 
      c_double(alpha), 
      byref(a), 
      c_int(lda), 
      byref(b), 
      c_int(ldb), 
      c_double(beta), 
      byref(c), 
      c_int(ldc))

386946416

In [6]:
# result in c
print_mat(c, 2, 4)

[0.0, 0.0, 0.0, 0.0]
--------------------
[0.0, 0.0, 0.0, 0.0]
--------------------
